## Step 1: Importing Libraries

In [1]:
import pandas as pd
import numpy as np

## Step 2: Load and Explore Data

In [2]:
retail = pd.read_csv("Book1.csv", encoding='unicode_escape')

In [3]:
retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [4]:
retail.shape

(541910, 8)

In [5]:
print("number of unique customers:", retail['Customer ID'].nunique())

number of unique customers: 4372


In [6]:
#checking duplicates
print(f"there are {retail.duplicated().sum()} duplicated rows")

there are 5268 duplicated rows


In [7]:
#viewing duplicated rows
retail[retail.duplicated()].head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,01-12-2010 11:45,1.25,17908.0,United Kingdom
527,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,01-12-2010 11:45,2.10,17908.0,United Kingdom
537,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,01-12-2010 11:45,2.95,17908.0,United Kingdom
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,01-12-2010 11:45,4.95,17908.0,United Kingdom
555,536412,22327,ROUND SNACK BOXES SET OF 4 SKULLS,1,01-12-2010 11:49,2.95,17920.0,United Kingdom
587,536412,22273,FELTCRAFT DOLL MOLLY,1,01-12-2010 11:49,2.95,17920.0,United Kingdom
589,536412,22749,FELTCRAFT PRINCESS CHARLOTTE DOLL,1,01-12-2010 11:49,3.75,17920.0,United Kingdom
594,536412,22141,CHRISTMAS CRAFT TREE TOP ANGEL,1,01-12-2010 11:49,2.10,17920.0,United Kingdom
598,536412,21448,12 DAISY PEGS IN WOOD BOX,1,01-12-2010 11:49,1.65,17920.0,United Kingdom
600,536412,22569,FELTCRAFT CUSHION BUTTERFLY,2,01-12-2010 11:49,3.75,17920.0,United Kingdom


Note: After manual review, there are indeed lots of duplicated rows (e.g. row 371 and 394). Let's remove them.

In [8]:
retail = retail.drop_duplicates(keep='first')

In [9]:
#checking missing values
retail.isna().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135037
Country             0
dtype: int64

Note: there are 234007 rows without Customer ID. Since we are working at the customer level, we cannot aggregate these columns. Let's remove them from further analysis.

In [10]:
retail = retail[retail['Customer ID'].notna()]

In [11]:
# checking types of columns
retail.dtypes

Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
dtype: object

## Step 3: Calculating Cohort Index

In [13]:
import datetime

In [16]:
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'], format='mixed')

In [17]:
retail['YearMonth'] = retail['InvoiceDate'].dt.strftime("%Y-%m")
retail['YearMonth'] = pd.to_datetime(retail['YearMonth'])

In [18]:
retail['CohortMonth'] = retail.groupby(by=['Customer ID'])['YearMonth'].transform('min')

In [19]:
def calculate_cohort_index(df, transactions_col, cohort_month):
    
    year_diff = df[transactions_col].dt.year - df[cohort_month].dt.year

    month_diff = df[transactions_col].dt.month - df[cohort_month].dt.month
    
    cohort_index = year_diff * 12 + month_diff + 1
    
    return cohort_index

In [20]:
retail['CohortIndex'] = calculate_cohort_index(retail, 'InvoiceDate', 'CohortMonth')

In [21]:
retail['CohortIndex'].max()

24

In [22]:
retail.tail()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,YearMonth,CohortMonth,CohortIndex
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-09-12 12:50:00,2.10,12680.0,France,2011-09-01,2011-08-01,2
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-09-12 12:50:00,4.15,12680.0,France,2011-09-01,2011-08-01,2
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-09-12 12:50:00,4.15,12680.0,France,2011-09-01,2011-08-01,2
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-09-12 12:50:00,4.95,12680.0,France,2011-09-01,2011-08-01,2
541909,581587,POST,POSTAGE,1,2011-09-12 12:50:00,18.00,12680.0,France,2011-09-01,2011-08-01,2


## Step 4: Calculating Average Active Users per Cohort

In [23]:
users_in_cohorts = retail.groupby(by=['CohortMonth', 'CohortIndex'])['Customer ID'].nunique()

In [24]:
users_in_cohorts = users_in_cohorts.reset_index()
users_in_cohorts.head()

,CohortMonth,CohortIndex,Customer ID
0,2010-01-01,1,98
1,2010-01-01,2,9
2,2010-01-01,3,4
3,2010-01-01,5,7
4,2010-01-01,6,7


In [25]:
cohorts_fin = users_in_cohorts.pivot(index='CohortMonth', columns='CohortIndex', values='Customer ID')
cohorts_fin

CohortIndex,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
CohortMonth,,,,,,,,,,,,,,,,,,,,,
2010-01-01,98.0,9.0,4.0,NaN,7.0,7.0,4.0,10.0,10.0,5.0,...,38.0,40.0,36.0,44.0,41.0,37.0,41.0,44.0,40.0,23.0
2010-02-01,108.0,2.0,NaN,NaN,3.0,7.0,5.0,12.0,5.0,NaN,...,39.0,38.0,43.0,48.0,34.0,41.0,45.0,53.0,19.0,NaN
2010-03-01,49.0,NaN,1.0,1.0,1.0,2.0,5.0,NaN,NaN,14.0,...,23.0,16.0,19.0,19.0,13.0,24.0,23.0,9.0,NaN,NaN
2010-05-01,68.0,3.0,3.0,2.0,3.0,1.0,NaN,24.0,32.0,20.0,...,30.0,32.0,31.0,33.0,40.0,20.0,NaN,NaN,NaN,NaN
2010-06-01,76.0,2.0,4.0,4.0,1.0,NaN,26.0,29.0,25.0,25.0,...,27.0,33.0,26.0,39.0,5.0,NaN,NaN,NaN,NaN,NaN
2010-07-01,53.0,2.0,2.0,1.0,NaN,9.0,24.0,22.0,22.0,19.0,...,22.0,20.0,29.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-01,87.0,3.0,2.0,NaN,30.0,37.0,30.0,34.0,33.0,36.0,...,41.0,49.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-01,83.0,1.0,NaN,23.0,29.0,20.0,30.0,23.0,29.0,26.0,...,32.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-01,38.0,NaN,11.0,15.0,12.0,15.0,18.0,8.0,14.0,11.0,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Step 5: Calculating Retention Rates

In [26]:
cohort_sizes = cohorts_fin.iloc[:, 0]
retention = cohorts_fin.divide(cohort_sizes, axis='rows').round(3)

In [27]:
retention

CohortIndex,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
CohortMonth,,,,,,,,,,,,,,,,,,,,,
2010-01-01,1.0,0.092,0.041,NaN,0.071,0.071,0.041,0.102,0.102,0.051,...,0.388,0.408,0.367,0.449,0.418,0.378,0.418,0.449,0.408,0.235
2010-02-01,1.0,0.019,NaN,NaN,0.028,0.065,0.046,0.111,0.046,NaN,...,0.361,0.352,0.398,0.444,0.315,0.380,0.417,0.491,0.176,NaN
2010-03-01,1.0,NaN,0.020,0.020,0.020,0.041,0.102,NaN,NaN,0.286,...,0.469,0.327,0.388,0.388,0.265,0.490,0.469,0.184,NaN,NaN
2010-05-01,1.0,0.044,0.044,0.029,0.044,0.015,NaN,0.353,0.471,0.294,...,0.441,0.471,0.456,0.485,0.588,0.294,NaN,NaN,NaN,NaN
2010-06-01,1.0,0.026,0.053,0.053,0.013,NaN,0.342,0.382,0.329,0.329,...,0.355,0.434,0.342,0.513,0.066,NaN,NaN,NaN,NaN,NaN
2010-07-01,1.0,0.038,0.038,0.019,NaN,0.170,0.453,0.415,0.415,0.358,...,0.415,0.377,0.547,0.113,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-01,1.0,0.034,0.023,NaN,0.345,0.425,0.345,0.391,0.379,0.414,...,0.471,0.563,0.253,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-01,1.0,0.012,NaN,0.277,0.349,0.241,0.361,0.277,0.349,0.313,...,0.386,0.120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-10-01,1.0,NaN,0.289,0.395,0.316,0.395,0.474,0.211,0.368,0.289,...,0.158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Step 6: Calculate Basic CLV (revenue-based)

**Basic CLV** = Average Revenue * Average Lifespan

In [28]:
retail['PurchaseSize'] = retail['Quantity']*retail['Price']

In [29]:
# calculate monthly spend per customer
monthly_revenue = retail.groupby(by=['Customer ID', 'YearMonth'])['PurchaseSize'].sum().mean()
monthly_revenue

587.5053171527926

Average Lifespan can be inferred by looking into the average time it takes for customers to churn from the time they made their first purchase. For now, we assume that the customer lifespan is 3 years or 36 months.

In [30]:
lifespan_months = 36

In [31]:
CLV_basic = monthly_revenue * lifespan_months
CLV_basic

21150.191417500533

Basic Customer Lifetime Value is 21725.62 USD.

## Step 7: Calculate Granular CLV (revenue-based)

**Granular CLV** = (Average Revenue per Transaction * Average Frequency) * Average Lifespan

In [32]:
# first mean() averages the revenue per invoice 
# second mean() averages the revenue per purchase
revenue_per_purchase = retail.groupby(by=['Invoice'])['PurchaseSize'].mean().mean()
revenue_per_purchase

34.385729012217254

In [33]:
# calculate average number of unique invoices per customer each month
avg_frequency_per_month = retail.groupby(by=['Customer ID', 'YearMonth'])['Invoice'].nunique().mean()
avg_frequency_per_month

1.5747640337804272

In [34]:
CLV_granular = revenue_per_purchase * avg_frequency_per_month * lifespan_months
CLV_granular

1949.3787356553566

Granular CLV is 1865.34 USD, with each customer making on average 1.66 transactions per month and spending about 31 USD per transaction.

## Step 8: Calculate Traditional CLV (revenue-based)

Traditional CLV method doens't require to define customer lifespan and instead uses retention to churn rate to access customer life expectancy.

**Traditional CLV** = Average Revenue * Retention Rate / Churn Rate

In [35]:
# calculate monthly spend per customer
monthly_revenue = retail.groupby(by=['Customer ID', 'YearMonth'])['PurchaseSize'].sum().mean()
monthly_revenue

587.5053171527926

In [36]:
#calculate average retention rate per customer
avg_retention = retention.iloc[:, 1:].mean().mean()
avg_retention

0.3063579887670024

In [37]:
avg_churn = 1 - avg_retention
avg_churn

0.6936420112329976

In [38]:
CLV_traditional = monthly_revenue * (avg_retention/avg_churn)
CLV_traditional

259.481035517024

In [40]:
print("Average CLV: {:.2f} USD at {:.1%} average retention rate and {:.2f} USD average monthly revenue".format(CLV_traditional, avg_retention, monthly_revenue))

Average CLV: 259.48 USD at 30.6% average retention rate and 587.51 USD average monthly revenue
